In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')
print(DEVICE)
BATCH_SIZE = 224

C:\Users\kyc_o\anaconda3\envs\dl\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")


cuda


In [2]:
class Net(nn.Module):
    def __init__(self):
        
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 32, 3, padding = 1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding = 1)
        self.conv3 = nn.Conv2d(64, 64, 3, padding = 1)
        
        self.fc1 = nn.Linear(4096, 512)
        self.fc2 = nn.Linear(512, 450)
        
    def forward(self, x):
        
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training)
        
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training)
        
        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training)
        
        x = x.view(-1, 4096)
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        
        return F.log_softmax(x, dim=1)

In [3]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            
            
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            
        test_loss /= len(test_loader.dataset)
        test_accuracy = 100. * correct / len(test_loader.dataset)
        return test_loss, test_accuracy

In [4]:
transform_base = transforms.Compose([transforms.Resize([64, 64]), transforms.ToTensor()])
test_dataset = ImageFolder(root='./dataset/test', transform=transform_base)
test_loader_base = torch.utils.data.DataLoader(test_dataset, batch_size = BATCH_SIZE, shuffle=True, num_workers=4)

In [24]:
transform_resNet = transforms.Compose([
    transforms.Resize([64, 64]),
    transforms.RandomCrop(52),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_resNet = ImageFolder(root='./dataset/test', transform=transform_resNet)

test_loader_resNet = torch.utils.data.DataLoader(test_resNet, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

In [5]:
transform_img_aug_1 = transforms.Compose([
    transforms.RandomCrop(170),
    transforms.Resize([64, 64]),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

transform_img_aug_23 = transforms.Compose([
    transforms.Resize([64, 64]),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

transform_img_aug_4 = transforms.Compose([
        transforms.RandomCrop(170),
        transforms.Resize([64, 64]),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

transform_img_aug_5 = transforms.Compose([
        transforms.Resize([224, 224]),
        transforms.RandomCrop(200),
        transforms.Resize([64, 64]),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

transform_img_aug_6 = transforms.Compose([
        transforms.Resize([224, 224]),
        transforms.RandomCrop(180),
        transforms.Resize([64, 64]),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_dataset_ia_1 = ImageFolder(root='./dataset/test', transform=transform_img_aug_1)
test_dataset_ia_23 = ImageFolder(root='./dataset/test', transform=transform_img_aug_23)
test_dataset_ia_4 = ImageFolder(root='./dataset/test', transform=transform_img_aug_4)
test_dataset_ia_5 = ImageFolder(root='./dataset/test', transform=transform_img_aug_5)
test_dataset_ia_6 = ImageFolder(root='./dataset/test', transform=transform_img_aug_6)

test_loader_ia_1 = torch.utils.data.DataLoader(test_dataset_ia_1, batch_size = BATCH_SIZE, shuffle=True, num_workers=4)
test_loader_ia_23 = torch.utils.data.DataLoader(test_dataset_ia_23, batch_size = BATCH_SIZE, shuffle=True, num_workers=4)
test_loader_ia_4 = torch.utils.data.DataLoader(test_dataset_ia_4, batch_size = BATCH_SIZE, shuffle=True, num_workers=4)
test_loader_ia_5 = torch.utils.data.DataLoader(test_dataset_ia_5, batch_size = BATCH_SIZE, shuffle=True, num_workers=4)
test_loader_ia_6 = torch.utils.data.DataLoader(test_dataset_ia_6, batch_size = BATCH_SIZE, shuffle=True, num_workers=4)

In [6]:
baseline=torch.load('./baseline/baseline.pt')
baseline.eval()
test_loss, test_accuracy = evaluate(baseline, test_loader_base)

print('baseline test accuracy: ', test_accuracy)

baseline test accuracy:  74.53333333333333


In [9]:
resnet50 = torch.load('./transfer/resnet50.pt')
resnet50.eval()
test_loss, test_accuracy = evaluate(resnet50, test_loader_resNet)

print('resNet test accuracy: ', test_accuracy)

resNet test accuracy:  93.15555555555555


In [12]:
baseline_pe=torch.load('./baseline_pumped_epoch/baseline_pumped_epoch.pt')
baseline_pe.eval()
test_loss, test_accuracy = evaluate(baseline_pe, test_loader_base)

print('baseline with more epochs test accuracy: ', test_accuracy)

baseline with more epochs test accuracy:  76.71111111111111


In [26]:
baseline_im1=torch.load('./baseline_img_aug_attempt1/baseline.pt')
baseline_im1.eval()
test_loss, test_accuracy = evaluate(baseline_im1, test_loader_ia_1)

print('img aug_v1 test accuracy: ', test_accuracy)

img aug_v1 test accuracy:  56.93333333333333


In [27]:
baseline_im2=torch.load('./baseline_img_aug_attempt2/baseline.pt')
baseline_im2.eval()
test_loss, test_accuracy = evaluate(baseline_im2, test_loader_ia_23)

print('img aug_v2 accuracy: ', test_accuracy)

img aug_v2 accuracy:  75.82222222222222


In [28]:
baseline_im3=torch.load('./baseline_img_aug_attempt3/baseline.pt')
baseline_im3.eval()
test_loss, test_accuracy = evaluate(baseline_im3, test_loader_ia_23)

print('img aug_v3 accuracy: ', test_accuracy)

img aug_v3 accuracy:  72.62222222222222


In [6]:
baseline_im4=torch.load('./baseline_img_aug_attempt4/baseline.pt')
baseline_im4.eval()
test_loss, test_accuracy = evaluate(baseline_im4, test_loader_ia_4)

print('img aug_v4 accuracy: ', test_accuracy)

img aug_v4 accuracy:  67.06666666666666


In [7]:
baseline_im5=torch.load('./baseline_img_aug_attempt5/baseline.pt')
baseline_im5.eval()
test_loss, test_accuracy = evaluate(baseline_im5, test_loader_ia_5)

print('img aug_v5 accuracy: ', test_accuracy)

img aug_v5 accuracy:  65.24444444444444


In [8]:
baseline_im6=torch.load('./baseline_img_aug_attempt6/baseline.pt')
baseline_im6.eval()
test_loss, test_accuracy = evaluate(baseline_im6, test_loader_ia_6)

print('img aug_v6 accuracy: ', test_accuracy)

img aug_v6 accuracy:  64.62222222222222


----

In [10]:
baseline=torch.load('./baseline_dropout1/baseline.pt')
baseline.eval()
test_loss, test_accuracy = evaluate(baseline, test_loader_ia_23)

print('dropout1 test accuracy: ', test_accuracy)

dropout1 test accuracy:  77.33333333333333


In [11]:
baseline=torch.load('./baseline_dropout2/baseline.pt')
baseline.eval()
test_loss, test_accuracy = evaluate(baseline, test_loader_ia_23)

print('dropout2 test accuracy: ', test_accuracy)

dropout2 test accuracy:  72.75555555555556


In [12]:
baseline=torch.load('./baseline_dropout3/baseline.pt')
baseline.eval()
test_loss, test_accuracy = evaluate(baseline, test_loader_ia_23)

print('dropout3 test accuracy: ', test_accuracy)

dropout3 test accuracy:  73.37777777777778


In [13]:
baseline=torch.load('./baseline_dropout4/baseline.pt')
baseline.eval()
test_loss, test_accuracy = evaluate(baseline, test_loader_ia_23)

print('dropout4 test accuracy: ', test_accuracy)

dropout4 test accuracy:  74.26666666666667


In [14]:
baseline=torch.load('./baseline_dropout5/baseline.pt')
baseline.eval()
test_loss, test_accuracy = evaluate(baseline, test_loader_ia_23)

print('dropout5 test accuracy: ', test_accuracy)

dropout5 test accuracy:  76.62222222222222


----

In [16]:
baseline=torch.load('./baseline_fc/baseline.pt')
baseline.eval()
test_loss, test_accuracy = evaluate(baseline, test_loader_base)

print('fc test accuracy: ', test_accuracy)

fc test accuracy:  74.88888888888889


In [18]:
baseline=torch.load('./baseline_lr/baseline.pt')
baseline.eval()
test_loss, test_accuracy = evaluate(baseline, test_loader_ia_23)

print('lr test accuracy: ', test_accuracy)

lr test accuracy:  75.55555555555556


In [19]:
baseline=torch.load('./baseline_leaky/baseline.pt')
baseline.eval()
test_loss, test_accuracy = evaluate(baseline, test_loader_ia_23)

print('leaky relu test accuracy: ', test_accuracy)

leaky relu test accuracy:  70.75555555555556
